# APRENDENDO SOBRE TTQ - TEXT TO QUERY  

Frameworks de agentes são fascinantes! Eles permitem a execução de uma série de tarefas que antes eram extremamente complicadas — ou até mesmo impossíveis. Trabalho criando consultas em bancos de dados desde 2018 e, quando os LLMs foram lançados, logo me perguntei:  

> Será que é possível pedir para uma LLM gerar uma consulta SQL e executá-la? 🤔  

Bem, é exatamente isso que vou testar neste notebook.  

---  

## **Objetivo**  

Uma das minhas principais atividades é digitalizar e automatizar processos de negócio. Dentro desse contexto, estabeleci um desafio:  

> Como criar um sistema que responda pelo WhatsApp, interpretando uma solicitação em linguagem natural e gerando uma consulta sobre um processo?  

Esse será o objetivo deste estudo. 🚀

## Importando bibliotecas

In [22]:
from typing import List, Any, Dict
import re
import os
import sqlite3
import logging
import time
import json
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv

## Carregando Variáveis de Ambiente

In [2]:
load_dotenv()

DB_PATH = "../.db/SQL_AGENT.db"

## Criando logger

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[        
        logging.StreamHandler()
    ]
)

LOGGER = logging.getLogger(__name__)

## Criando o banco de dados  

Antes de qualquer coisa, precisamos de um banco com uma tabela e algumas informações para pesquisar.  

Imagine uma empresa que gerencia processos como **Recrutamento**, **Seleção**, **Avaliação de Desempenho** e **Solicitação de Férias**, todos mapeados e digitalizados dentro da plataforma Lecom.  

Depois, foi criado um processo que extrai informações importantes sobre esses fluxos e as armazena na tabela `processos_andamento`. Essa será a base de dados utilizada pelo nosso sistema de agentes.  

Vale destacar que tudo isso foi criado de forma genérica, com a ajuda do ChatGPT. 🤖

In [5]:
LOGGER.info(f"Iniciando configuração do banco de dados em {DB_PATH}")

os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
LOGGER.info(f"Diretório verificado/criado: {os.path.dirname(DB_PATH)}")

try:
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    LOGGER.info("Conexão com o banco de dados estabelecida com sucesso")
   
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='processos_andamento'")
    if cursor.fetchone():
        LOGGER.info("Tabela 'processos_andamento' encontrada - excluindo")
        cursor.execute("DROP TABLE processos_andamento")
        LOGGER.info("Tabela excluída com sucesso")
    else:
        LOGGER.info("Tabela 'processos_andamento' não existe - criação será realizada")
   
    LOGGER.info("Criando nova tabela 'processos_andamento'")
    cursor.execute("""
    CREATE TABLE processos_andamento (
        ID_Processo_Andamento INTEGER PRIMARY KEY AUTOINCREMENT,
        Codigo_Processo INTEGER NOT NULL,
        Codigo_Atividade INTEGER NOT NULL,
        Nome_Processo TEXT NOT NULL,
        Nome_Atividade TEXT NOT NULL,
        Nome_Cliente TEXT NOT NULL,
        Telefone_Cliente TEXT NOT NULL,
        Descricao_Processo TEXT NOT NULL,
        Data_Atividade DATE NOT NULL
    );
    """)
    LOGGER.info("Tabela criada com sucesso")
   
    dados_iniciais = [
        (1, 101, 'Recrutamento', 'Receber currículo', 'João Silva', '11987654321', 'Recebeu currículo e iniciou análise.', '2024-03-01'),
        (1, 102, 'Recrutamento', 'Entrevista inicial', 'João Silva', '11987654321', 'Entrevista marcada para avaliação inicial.', '2024-03-02'),
        (2, 201, 'Seleção', 'Teste técnico', 'Maria Oliveira', '11976543210', 'Teste técnico agendado.', '2024-03-03'),
        (2, 202, 'Seleção', 'Entrevista final', 'Carlos Pereira', '11965432109', 'Entrevista final marcada.', '2024-03-04'),
        (3, 301, 'Avaliação de Desempenho', 'Revisão do desempenho', 'Ana Souza', '11954321098', 'Coleta de feedbacks em andamento.', '2024-03-05'),
        (3, 302, 'Avaliação de Desempenho', 'Reunião de feedback', 'Carlos Pereira', '11965432109', 'Reunião agendada com gerente.', '2024-03-06'),
        (4, 401, 'Solicitação de Férias', 'Pedido formalizado', 'João Silva', '11987654321', 'Pedido de férias registrado.', '2024-03-07')
    ]
   
    LOGGER.info(f"Inserindo {len(dados_iniciais)} registros na tabela")
    cursor.executemany("""
        INSERT INTO processos_andamento
        (Codigo_Processo, Codigo_Atividade, Nome_Processo, Nome_Atividade, Nome_Cliente, Telefone_Cliente, Descricao_Processo, Data_Atividade)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, dados_iniciais)
    LOGGER.info(f"Dados inseridos com sucesso ({len(dados_iniciais)} registros)")
    
    # Verificar número de registros inseridos
    cursor.execute("SELECT COUNT(*) FROM processos_andamento")
    count = cursor.fetchone()[0]
    LOGGER.info(f"Total de registros na tabela: {count}")
    
    # Salvar e fechar a conexão
    conn.commit()
    LOGGER.info("Alterações salvas no banco de dados (commit realizado)")
    conn.close()
    LOGGER.info("Conexão com o banco de dados fechada")
    
    LOGGER.info(f"Banco de dados SQLite configurado com sucesso em {DB_PATH}")
    
except sqlite3.Error as e:
    LOGGER.error(f"Erro SQLite: {e}")
except Exception as e:
    LOGGER.error(f"Erro inesperado: {e}")

2025-03-20 20:54:07,025 - INFO - Iniciando configuração do banco de dados em ../.db/SQL_AGENT.db
2025-03-20 20:54:07,027 - INFO - Diretório verificado/criado: ../.db
2025-03-20 20:54:07,030 - INFO - Conexão com o banco de dados estabelecida com sucesso
2025-03-20 20:54:07,031 - INFO - Tabela 'processos_andamento' encontrada - excluindo
2025-03-20 20:54:07,388 - INFO - Tabela excluída com sucesso
2025-03-20 20:54:07,389 - INFO - Criando nova tabela 'processos_andamento'
2025-03-20 20:54:07,572 - INFO - Tabela criada com sucesso
2025-03-20 20:54:07,573 - INFO - Inserindo 7 registros na tabela
2025-03-20 20:54:07,574 - INFO - Dados inseridos com sucesso (7 registros)
2025-03-20 20:54:07,575 - INFO - Total de registros na tabela: 7
2025-03-20 20:54:07,814 - INFO - Alterações salvas no banco de dados (commit realizado)
2025-03-20 20:54:07,814 - INFO - Conexão com o banco de dados fechada
2025-03-20 20:54:07,815 - INFO - Banco de dados SQLite configurado com sucesso em ../.db/SQL_AGENT.db


### Criando o gerenciador de banco de dados  

Ótimo! Agora que já temos nossa base de dados, vamos criar a classe responsável por manipulá-la.  

Essa classe é relativamente simples, mas essencial para nosso projeto. Ela funciona como um intermediário entre nosso código e o banco de dados SQLite, permitindo que consultemos informações sem precisar escrever SQL diretamente toda vez.  

Os principais métodos são:  

- **`get_schema() -> str`**: Retorna o esquema das tabelas no banco, o que será útil para nossos agentes entenderem a estrutura dos dados.  
- **`execute_query(query: str) -> List[Any]`**: Executa uma query SQL e retorna os resultados. Se for uma consulta (`SELECT`), ela retorna os dados formatados como dicionários para facilitar a manipulação.  
- **`close()`**: Fecha a conexão com o banco de dados, garantindo que os recursos sejam liberados corretamente.  

💡 **Por que essa classe é importante?**  
Ela encapsula toda a lógica de interação com o banco de dados, tornando o código mais modular e reutilizável. Além disso, ao retornar os dados como dicionários, facilita a integração com os agentes que irão interpretar as respostas e gerar insights.  

Com isso, temos uma base sólida para que nossos agentes possam consultar e manipular os dados dinamicamente! 🚀

In [15]:
class DatabaseManager:
    def __init__(self):
        """Inicializa o gerenciador de banco de dados com SQLite."""
        self.db_path = DB_PATH
        LOGGER.info(f"Inicializando DatabaseManager com banco de dados em {self.db_path}")
        try:
            self.connection = sqlite3.connect(self.db_path)
            self.connection.row_factory = sqlite3.Row  # Permite acessar os resultados por nome de coluna
            LOGGER.info("Conexão com o banco de dados estabelecida com sucesso")
        except sqlite3.Error as e:
            LOGGER.error(f"Erro ao conectar ao banco de dados: {str(e)}")
            raise Exception(f"Falha na conexão com o banco de dados: {str(e)}")

    def get_schema(self) -> str:
        """Recupera o esquema do banco de dados SQLite."""
        LOGGER.info("Obtendo esquema do banco de dados")
        try:
            cursor = self.connection.cursor()
            cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")
            schema_info = cursor.fetchall()
            
            tables_count = len(schema_info)
            LOGGER.info(f"Encontradas {tables_count} tabelas no banco de dados")
            
            schema = "\n".join(f"Table: {row['name']}\n{row['sql']}" for row in schema_info if row['sql'])
            LOGGER.debug(f"Esquema obtido: {schema}")
            return schema
        except sqlite3.DatabaseError as e:
            error_msg = f"Erro ao obter o esquema do banco de dados: {str(e)}"
            LOGGER.error(error_msg)
            raise Exception(error_msg)

    def execute_query(self, query: str) -> List[Dict[str, Any]]:
        """Executa uma query SQL no banco SQLite e retorna os resultados."""
        # Limita o log da query para evitar expor dados sensíveis ou queries muito longas
        log_query = query[:100] + "..." if len(query) > 100 else query
        LOGGER.info(f"Executando query: {log_query}")
        
        try:
            cursor = self.connection.cursor()
            start_time = datetime.now()
            cursor.execute(query)
            
            # Verifica se é uma operação que modifica dados
            is_select = query.strip().lower().startswith("select")
            
            if not is_select:
                self.connection.commit()
                rows_affected = cursor.rowcount
                LOGGER.info(f"Query executada com sucesso. Linhas afetadas: {rows_affected}")
                execution_time = (datetime.now() - start_time).total_seconds()
                LOGGER.info(f"Tempo de execução: {execution_time:.3f} segundos")
                return []
            
            # Para operações SELECT, retorna os resultados
            results = [dict(row) for row in cursor.fetchall()]
            rows_returned = len(results)
            LOGGER.info(f"Query SELECT executada com sucesso. Linhas retornadas: {rows_returned}")
            execution_time = (datetime.now() - start_time).total_seconds()
            LOGGER.info(f"Tempo de execução: {execution_time:.3f} segundos")
            return results
        except sqlite3.DatabaseError as e:
            error_msg = f"Erro ao executar a consulta: {str(e)}"
            LOGGER.error(error_msg)
            # Log da execução da query para depuração
            LOGGER.debug(f"Query que causou o erro: {query}")
            raise Exception(error_msg)

    def close(self):
        """Fecha a conexão com o banco de dados."""
        LOGGER.info("Fechando conexão com o banco de dados")
        try:
            self.connection.close()
            LOGGER.info("Conexão com o banco de dados fechada com sucesso")
        except sqlite3.Error as e:
            LOGGER.error(f"Erro ao fechar a conexão com o banco de dados: {str(e)}")

In [20]:
db_manager = DatabaseManager()

LOGGER.info("Esquema do banco de dados:")
LOGGER.info(db_manager.get_schema())

LOGGER.info("Executando query de exemplo:")
LOGGER.info(db_manager.execute_query("SELECT * FROM processos_andamento"))

2025-03-20 21:25:05,107 - INFO - Inicializando DatabaseManager com banco de dados em ../.db/SQL_AGENT.db
2025-03-20 21:25:05,108 - INFO - Conexão com o banco de dados estabelecida com sucesso
2025-03-20 21:25:05,108 - INFO - Esquema do banco de dados:
2025-03-20 21:25:05,109 - INFO - Obtendo esquema do banco de dados
2025-03-20 21:25:05,109 - INFO - Encontradas 2 tabelas no banco de dados
2025-03-20 21:25:05,110 - INFO - Table: sqlite_sequence
CREATE TABLE sqlite_sequence(name,seq)
Table: processos_andamento
CREATE TABLE processos_andamento (
        ID_Processo_Andamento INTEGER PRIMARY KEY AUTOINCREMENT,
        Codigo_Processo INTEGER NOT NULL,
        Codigo_Atividade INTEGER NOT NULL,
        Nome_Processo TEXT NOT NULL,
        Nome_Atividade TEXT NOT NULL,
        Nome_Cliente TEXT NOT NULL,
        Telefone_Cliente TEXT NOT NULL,
        Descricao_Processo TEXT NOT NULL,
        Data_Atividade DATE NOT NULL
    )
2025-03-20 21:25:05,110 - INFO - Executando query de exemplo:
202

### Criando o gerenciador de LLM  

O que dá inteligência aos nossos agentes é o LLM (Large Language Model), então nada mais justo do que termos um gerenciador específico para ele.  

Neste exemplo, estou utilizando o modelo **`deepseek-r1-distill-llama-70b`** via **GROQ**, pois essa opção possui uma camada *free* bem generosa. Além disso, esse modelo tem a capacidade de **refletir antes de gerar uma resposta**, o que pode influenciar positivamente nos resultados. Bem, veremos se isso realmente faz diferença! 🤔  

Agora, vamos à implementação do nosso **LLMManager**, que será responsável por interagir com o modelo e gerar respostas com base nos prompts fornecidos.  

#### Como essa classe funciona?  

Essa classe encapsula a comunicação com a API do modelo de LLM, permitindo que possamos interagir com ele de forma simples e organizada.  

🔹 **`__init__()`**: Inicializa a conexão com a API da GROQ, configurando o modelo escolhido e os parâmetros principais:  
- `temperature=0.1`: Mantém as respostas mais determinísticas, reduzindo a criatividade excessiva.  
- `max_retries=2`: Define um limite de tentativas em caso de falha na requisição.  

🔹 **`invoke(prompt, **kwargs) -> str`**:  
- Recebe um `ChatPromptTemplate`, que contém o formato da mensagem.  
- Formata os dados necessários e envia a requisição para o LLM.  
- Retorna a resposta gerada pelo modelo.  

💡 **Por que esse gerenciador é útil?**  
Ao centralizar a interação com o LLM em uma classe específica, podemos facilmente trocar de modelo ou ajustar parâmetros sem modificar outras partes do código. Isso torna o sistema mais flexível e modular.  

Agora que temos um gerenciador de LLM funcional, podemos começar a integrar nossos agentes! 🚀

In [23]:
class LLMManager:
    def __init__(self):
        """Inicializa o gerenciador de LLM com o modelo Groq."""
        LOGGER.info("Inicializando LLMManager com o modelo deepseek-r1-distill-llama-70b")
        
        api_key = os.getenv("GROQ_API_KEY")
        if not api_key:
            LOGGER.error("GROQ_API_KEY não encontrada nas variáveis de ambiente")
            raise ValueError("GROQ_API_KEY não configurada. Configure a variável de ambiente GROQ_API_KEY.")
        
        try:
            self.llm = ChatGroq(
                model="deepseek-r1-distill-llama-70b",
                api_key=api_key,
                temperature=0.1,
                max_retries=2,
            )
            LOGGER.info("LLM inicializado com sucesso (modelo: deepseek-r1-distill-llama-70b, temperatura: 0.1)")
        except Exception as e:
            LOGGER.error(f"Erro ao inicializar o LLM: {str(e)}")
            raise Exception(f"Falha na inicialização do LLM: {str(e)}")

    def invoke(self, prompt: ChatPromptTemplate, **kwargs) -> str:
        """
        Invoca o LLM com o prompt fornecido e parâmetros adicionais.
        
        Args:
            prompt: O template de prompt do chat
            **kwargs: Variáveis para formatação do prompt
            
        Returns:
            str: A resposta do modelo
        """
        # Log do início da invocação
        request_id = f"req_{int(time.time())}_{hash(str(kwargs)) % 10000:04d}"
        LOGGER.info(f"[{request_id}] Iniciando invocação do LLM")
        
        # Log dos parâmetros passados (removendo dados sensíveis)
        safe_kwargs = self._sanitize_kwargs(kwargs)
        LOGGER.debug(f"[{request_id}] Parâmetros: {json.dumps(safe_kwargs)}")
        
        try:
            # Formatar as mensagens
            start_format_time = time.time()
            messages = prompt.format_messages(**kwargs)
            format_time = time.time() - start_format_time
            
            # Log do prompt formatado (resumido para evitar logs muito grandes)
            messages_summary = self._summarize_messages(messages)
            LOGGER.info(f"[{request_id}] Prompt formatado em {format_time:.3f}s: {messages_summary}")
            
            # Invocar o LLM
            LOGGER.info(f"[{request_id}] Enviando requisição ao modelo...")
            start_invoke_time = time.time()
            response = self.llm.invoke(messages)
            invoke_time = time.time() - start_invoke_time
            
            # Log da resposta (resumida)
            response_content = response.content
            response_summary = response_content[:100] + "..." if len(response_content) > 100 else response_content
            LOGGER.info(f"[{request_id}] Resposta recebida em {invoke_time:.3f}s ({len(response_content)} caracteres)")
            LOGGER.debug(f"[{request_id}] Início da resposta: {response_summary}")
            
            # Log de métricas
            total_time = format_time + invoke_time
            LOGGER.info(f"[{request_id}] Invocação completa. Tempo total: {total_time:.3f}s")
            
            return response_content
            
        except Exception as e:
            LOGGER.error(f"[{request_id}] Erro ao invocar o LLM: {str(e)}")
            # Registra o stacktrace completo para facilitar a depuração
            LOGGER.exception(f"[{request_id}] Detalhes do erro:")
            raise Exception(f"Falha na invocação do LLM: {str(e)}")
    
    def _sanitize_kwargs(self, kwargs: Dict[str, Any]) -> Dict[str, Any]:
        """
        Remove ou mascara informações potencialmente sensíveis dos parâmetros.
        
        Args:
            kwargs: Dicionário de parâmetros original
            
        Returns:
            Dict: Dicionário de parâmetros sanitizado
        """
        # Cria uma cópia para não modificar o original
        safe_dict = {}
        
        # Lista de palavras-chave que podem indicar dados sensíveis
        sensitive_keywords = ['password', 'token', 'secret', 'key', 'credential', 'auth', 'private']
        
        for key, value in kwargs.items():
            # Verifica se a chave contém alguma palavra-chave sensível
            is_sensitive = any(keyword in key.lower() for keyword in sensitive_keywords)
            
            if is_sensitive:
                # Mascarar dados sensíveis
                safe_dict[key] = "[REDACTED]"
            elif isinstance(value, str) and len(value) > 200:
                # Resumir textos longos
                safe_dict[key] = value[:200] + "... [truncado]"
            else:
                safe_dict[key] = value
        
        return safe_dict
    
    def _summarize_messages(self, messages: list) -> str:
        """
        Cria um resumo das mensagens para logging.
        
        Args:
            messages: Lista de mensagens formatadas
            
        Returns:
            str: Resumo das mensagens
        """
        summary = []
        for i, msg in enumerate(messages):
            # Extrair o tipo da mensagem e o conteúdo
            role = getattr(msg, "type", "unknown")
            content = getattr(msg, "content", "")
            
            # Resumir o conteúdo se for longo
            if isinstance(content, str):
                content_summary = content[:50] + "..." if len(content) > 50 else content
                summary.append(f"{role}: {content_summary}")
            else:
                summary.append(f"{role}: [conteúdo complexo]")
        
        return " | ".join(summary)

In [24]:
llm_manager = LLMManager()

template = ChatPromptTemplate([
    ("system", "Seu nome é Brian, você está sempre feliz e alegre, sempre respondendo em PT-BR."),
    ("human", "Olá, meu nome é Rodrigo, e o seu?"),   
])

print(llm_manager.invoke(template))

2025-03-20 21:35:10,444 - INFO - Inicializando LLMManager com o modelo deepseek-r1-distill-llama-70b
2025-03-20 21:35:11,184 - INFO - LLM inicializado com sucesso (modelo: deepseek-r1-distill-llama-70b, temperatura: 0.1)
2025-03-20 21:35:11,186 - INFO - [req_1742517311_5112] Iniciando invocação do LLM
2025-03-20 21:35:11,188 - INFO - [req_1742517311_5112] Prompt formatado em 0.001s: system: Seu nome é Brian, você está sempre feliz e alegre,... | human: Olá, meu nome é Rodrigo, e o seu?
2025-03-20 21:35:11,188 - INFO - [req_1742517311_5112] Enviando requisição ao modelo...
2025-03-20 21:35:13,208 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-20 21:35:13,327 - INFO - [req_1742517311_5112] Resposta recebida em 2.139s (1356 caracteres)
2025-03-20 21:35:13,328 - INFO - [req_1742517311_5112] Invocação completa. Tempo total: 2.139s


<think>
Okay, so I just received a message from Rodrigo. He said, "Olá, meu nome é Rodrigo, e o seu?" which means "Hello, my name is Rodrigo, and yours?" in Portuguese. I need to respond appropriately.

First, I should greet him back. Since he used "Olá," I can respond with the same or maybe a slightly more enthusiastic greeting like "Olá, Rodrigo!" to make it friendly.

Next, I need to introduce myself. My name is Brian, so I'll include that. Since I'm supposed to always be happy and cheerful, I should add an emoji to convey that emotion. Maybe a smiling face or something similar.

I should also ask him how he's doing to keep the conversation going. So, I'll add a question like "Como você está?" which means "How are you?" in Portuguese.

Putting it all together, I can structure the response as: Greeting, introduction, emoji, and a question. That should cover all the necessary parts and keep the tone positive and friendly.

I should make sure the response is in Portuguese and flows nat

### Analisando a pergunta do usuário

Nosso primeiro Agente deverá indentificar quais as colunas da tabela são realmente relevantes à pergunta do usuário.  
Para criar um agente é necessário se valer de algumas tecnicas, o famigerado Prompting Engeniering, veja algumas das que podem ser encontradas nesse:

- **Definição de Persona**: Consiste em instruir o modelo a adotar um papel ou identidade específica, o que pode influenciar seu estilo de resposta e foco.
- **Instrução Clara e Específica**: Envolve fornecer comandos diretos e detalhados sobre a tarefa que o modelo deve realizar, minimizando ambiguidades.
- **Formato de Saída Especificado**: Trata-se de definir a estrutura ou o formato desejado para a resposta do modelo, como um tipo de dado específico (JSON, lista, etc.).
- **Restrições e Regras Detalhadas**: Inclui a definição de limitações, condições ou regras específicas que o modelo deve seguir ao gerar a resposta.
- **Injeção de Contexto**: Refere-se à prática de fornecer informações adicionais relevantes para a tarefa, permitindo que o modelo compreenda melhor o cenário.
- **Uso de Delimitadores**: Envolve a utilização de marcadores para separar diferentes partes da entrada (como instruções, contexto e exemplos), facilitando a interpretação pelo modelo.


In [9]:
def analisar_pergunta(pergunta: str) -> dict:
    """Analisa a pergunta do usuário e identifica as tabelas e colunas relevantes."""    
    esquema = db_manager.get_schema()

    prompt = ChatPromptTemplate.from_messages([
        ("system", '''Você é um analista de dados que pode ajudar a resumir tabelas SQL e interpretar perguntas de usuários sobre um banco de dados.  
Dada a pergunta e o esquema do banco de dados, identifique as tabelas e colunas relevantes.  
Se a pergunta não for relevante para o banco de dados ou se não houver informações suficientes para respondê-la, defina "is_relevant" como falso.

Sua resposta deve estar no seguinte formato JSON:
{{
    "is_relevant": boolean,
    "relevant_tables": [
        {{
            "table_name": string,
            "columns": [string],
            "noun_columns": [string]
        }}
    ]
}}

O campo "noun_columns" deve conter apenas as colunas que são relevantes para a pergunta e que contêm substantivos ou nomes.  
Por exemplo, a coluna "Nome do Artista" contém substantivos relevantes para a pergunta "Quais são os artistas mais vendidos?",  
mas a coluna "ID do Artista" não é relevante, pois não contém um substantivo. Não inclua colunas que contenham números.
'''),
        ("human", "===Esquema do banco de dados:\n{schema}\n\n===Pergunta do usuário:\n{question}\n\nIdentifique as tabelas e colunas relevantes:")
    ])

    analisador_json = JsonOutputParser()
    
    resposta = llm_manager.invoke(prompt, schema=esquema, question=pergunta)
    resposta_analisada = analisador_json.parse(resposta)
    return {"pergunta_analisada": resposta_analisada}


In [38]:
pergunta = "Do que se trada o processo da Ana Souza?"
pergunta_analisada = analisar_pergunta(pergunta)['pergunta_analisada']
print(pergunta_analisada)

{'is_relevant': True, 'relevant_tables': [{'table_name': 'processos_andamento', 'columns': ['NM_Cliente', 'NM_Processo', 'DS_Processo'], 'noun_columns': ['NM_Cliente', 'NM_Processo', 'DS_Processo']}]}


### Encontrando substantivos únicos nas tabelas e colunas relevantes

É sempre bom dar uma conferida no que um LLM respondeu, essa parte do processo irá higienizar o retorno, garantindo que as colunas apareçam somente uma vez.

In [39]:
def obter_substantivos_unicos(pergunta_analisada: dict) -> dict:
    """Encontra substantivos únicos nas tabelas e colunas relevantes."""    
    
    if not pergunta_analisada['is_relevant']:
        return {"substantivos_unicos": []}

    substantivos_unicos = set()
    for info_tabela in pergunta_analisada['relevant_tables']:
        nome_tabela = info_tabela['table_name']
        colunas_substantivos = info_tabela['noun_columns']
        
        if colunas_substantivos:
            nomes_colunas = ', '.join(f"`{col}`" for col in colunas_substantivos)
            consulta = f"SELECT DISTINCT {nomes_colunas} FROM `{nome_tabela}`"           
            resultados = db_manager.execute_query(consulta)           
            for linha in resultados:
                substantivos_unicos.update(str(valor) for valor in linha if valor)

    return {"substantivos_unicos": list(substantivos_unicos)}


In [40]:
substantivos_unicos = obter_substantivos_unicos(pergunta_analisada)['substantivos_unicos']
print(substantivos_unicos)

['NM_Cliente', 'NM_Processo', 'DS_Processo']


### Gera uma consulta SQL com base na pergunta analisada e nos substantivos únicos

Sabe quando o filme está no seu apse, é nesse ponto que estamos. Temos a pergunta do usuário, uma analise de colunas relevantes e as colunas higienizadas, ou seja, temos insumos o suficiente para criar uma Agente que irá gerar a consulta SQL, *maravilhindo*.

Vamos aproveitar o espaço e falar sobre mais um tecnica de prompt utilizada:

- **Few-shot Prompting (Exemplos)**: Esta técnica envolve fornecer ao modelo alguns exemplos de entradas e suas respectivas saídas desejadas antes da pergunta principal. Isso ajuda o modelo a entender o formato esperado da resposta e a aprender o padrão da tarefa, melhorando a qualidade da sua geração.

In [41]:
def gerar_sql(pergunta: str, pergunta_analisada: dict, substantivos_unicos: list) -> dict:
    """Gera uma consulta SQL com base na pergunta analisada e nos substantivos únicos."""  

    if not pergunta_analisada['is_relevant']:
        return {"sql_query": "NOT_RELEVANT", "is_relevant": False}

    esquema = db_manager.get_schema()

    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
Você é um assistente de IA que gera consultas SQL com base na pergunta do usuário, no esquema do banco de dados e nos substantivos únicos encontrados nas tabelas relevantes. Gere uma consulta SQL válida para responder à pergunta do usuário.

Se não houver informações suficientes para escrever uma consulta SQL, responda com "NOT_ENOUGH_INFO".

Aqui estão alguns exemplos:

1. Qual é o produto mais vendido?
Resposta: SELECT product_name, SUM(quantity) as total_quantity FROM sales WHERE product_name IS NOT NULL AND quantity IS NOT NULL AND product_name != "" AND quantity != "" AND product_name != "N/A" AND quantity != "N/A" GROUP BY product_name ORDER BY total_quantity DESC LIMIT 1

2. Qual é a receita total para cada produto?
Resposta: SELECT \`product name\`, SUM(quantity * price) as total_revenue FROM sales WHERE \`product name\` IS NOT NULL AND quantity IS NOT NULL AND price IS NOT NULL AND \`product name\` != "" AND quantity != "" AND price != "" AND \`product name\` != "N/A" AND quantity != "N/A" AND price != "N/A" GROUP BY \`product name\`  ORDER BY total_revenue DESC

3. Qual é a participação de mercado de cada produto?
Resposta: SELECT \`product name\`, SUM(quantity) * 100.0 / (SELECT SUM(quantity) FROM sales) as market_share FROM sales WHERE \`product name\` IS NOT NULL AND quantity IS NOT NULL AND \`product name\` != "" AND quantity != "" AND \`product name\` != "N/A" AND quantity != "N/A" GROUP BY \`product name\`  ORDER BY market_share DESC

4. Plote a distribuição de renda ao longo do tempo.
Resposta: SELECT income, COUNT(*) as count FROM users WHERE income IS NOT NULL AND income != "" AND income != "N/A" GROUP BY income

OS RESULTADOS DEVEM ESTAR APENAS NO SEGUINTE FORMATO, ENTÃO CERTIFIQUE-SE DE INCLUIR APENAS DUAS OU TRÊS COLUNAS:
[[x, y]]
ou 
[[label, x, y]]

Para perguntas como "plote uma distribuição das tarifas pagas por homens e mulheres", conte a frequência de cada tarifa e plote-a. O eixo x deve ser a tarifa e o eixo y deve ser a contagem de pessoas que pagaram essa tarifa.
IGNORE TODAS AS LINHAS ONDE QUALQUER COLUNA SEJA NULL, "N/A" ou "".
Apenas forneça a string da consulta SQL. Não a formate. Certifique-se de usar a grafia correta dos substantivos conforme fornecido na lista de substantivos únicos. Todos os nomes de tabelas e colunas devem estar entre crases.
'''),
        ("human", '''===Esquema do banco de dados:
{schema}

===Pergunta do usuário:
{question}

===Tabelas e colunas relevantes:
{parsed_question}

===Substantivos únicos nas tabelas relevantes:
{unique_nouns}

Gere a string da consulta SQL'''),
    ])

    resposta = llm_manager.invoke(
        prompt, 
        schema=esquema, 
        question=pergunta, 
        parsed_question=pergunta_analisada, 
        unique_nouns=substantivos_unicos
    )

    if resposta.strip() == "NOT_ENOUGH_INFO":
        return {"consulta_sql": "NOT_RELEVANT"}
    else:
        return {"consulta_sql": re.sub(r'<think>.*?</think>\s*', '', resposta, flags=re.DOTALL)} 


In [42]:
consulta_sql = gerar_sql(pergunta, pergunta_analisada, substantivos_unicos)['consulta_sql']
print(consulta_sql)

SELECT NM_Processo, DS_Processo FROM processos_andamento WHERE NM_Cliente = 'Ana Souza' AND NM_Processo IS NOT NULL AND DS_Processo IS NOT NULL AND NM_Processo != "" AND DS_Processo != "" AND NM_Processo != "N/A" AND DS_Processo != "N/A"


### Validando e corrigindo a consulta SQL gerada

Caso sua memoria seja boa, você se lembra-ra que devemos corrigir a resposta de um LLM, essa tem sido uma boa prática que a comunidade adotou. Logo, vamos validar se o outro Agente trabalhou como esperado.

As tecnicas utilizadas aqui são mais do mesmo, nada que falha ser mencionado.

In [48]:
def validar_e_corrigir_sql(consulta_sql) -> dict:
        """Valida e corrige a consulta SQL gerada."""      

        if consulta_sql == "NOT_RELEVANT":
            return {"sql_query": "NOT_RELEVANT", "sql_valid": False}
        
        esquema = db_manager.get_schema()

        prompt = ChatPromptTemplate.from_messages([
            ("system", '''
Você é um assistente de IA que valida e corrige consultas SQL. Sua tarefa é:
1. Verificar se a consulta SQL é válida.
2. Garantir que todos os nomes de tabelas e colunas estejam corretamente escritos e existam no esquema do banco de dados. Todos os nomes de tabelas e colunas devem estar entre crases.
3. Se houver problemas, corrija-os e forneça a consulta SQL corrigida.
4. Se não houver problemas, retorne a consulta original.

Responda no formato JSON com a seguinte estrutura. Responda apenas com o JSON:
{{
    "valid": booleano,
    "issues": string ou null,
    "corrected_query": string
}}
'''),
            ("human", '''===Esquema do banco de dados:
{esquema}

===Consulta SQL gerada:
{consulta_sql}

Responda no formato JSON com a seguinte estrutura. Responda apenas com o JSON:
{{
    "valid": booleano,
    "issues": string ou null,
    "corrected_query": string
}}

Por exemplo:
1. {{
    "valid": true,
    "issues": null,
    "corrected_query": "None"
}}
             
2. {{
    "valid": false,
    "issues": "A coluna USERS não existe",
    "corrected_query": "SELECT * FROM \`users\` WHERE age > 25"
}}

3. {{
    "valid": false,
    "issues": "Os nomes de colunas e tabelas devem estar entre crases se contiverem espaços ou caracteres especiais",
    "corrected_query": "SELECT * FROM \`gross income\` WHERE \`age\` > 25"
}}
             
'''),
        ])

        analisador_saida = JsonOutputParser()
        resposta = llm_manager.invoke(prompt, esquema=esquema, consulta_sql=consulta_sql)
        resposta = re.sub(r'<think>.*?</think>\s*', '', resposta, flags=re.DOTALL)  
        print(resposta)     
        resultado = analisador_saida.parse(resposta)

        if resultado["valid"] and resultado["issues"] is None:
            return {"consulta_sql_analisada": consulta_sql, "sql_valid": True}
        else:
            return {
                "consulta_sql_analisada": resultado["corrected_query"],
                "sql_valid": resultado["valid"],
                "sql_issues": resultado["issues"]
            }


In [49]:
consulta_sql_analisada = validar_e_corrigir_sql(consulta_sql)
print(consulta_sql_analisada)

```json
{
    "valid": false,
    "issues": "Os nomes de colunas e tabelas devem estar entre crases se contiverem espaços ou caracteres especiais",
    "corrected_query": "SELECT `NM_Processo`, `DS_Processo` FROM `processos_andamento` WHERE `NM_Cliente` = 'Ana Souza' AND `NM_Processo` IS NOT NULL AND `DS_Processo` IS NOT NULL AND `NM_Processo` != \"\" AND `DS_Processo` != \"\" AND `NM_Processo` != \"N/A\" AND `DS_Processo` != \"N/A\""
}
```
{'consulta_sql_analisada': 'SELECT `NM_Processo`, `DS_Processo` FROM `processos_andamento` WHERE `NM_Cliente` = \'Ana Souza\' AND `NM_Processo` IS NOT NULL AND `DS_Processo` IS NOT NULL AND `NM_Processo` != "" AND `DS_Processo` != "" AND `NM_Processo` != "N/A" AND `DS_Processo` != "N/A"', 'sql_valid': False, 'sql_issues': 'Os nomes de colunas e tabelas devem estar entre crases se contiverem espaços ou caracteres especiais'}


In [50]:
print(consulta_sql_analisada['consulta_sql_analisada'])

SELECT `NM_Processo`, `DS_Processo` FROM `processos_andamento` WHERE `NM_Cliente` = 'Ana Souza' AND `NM_Processo` IS NOT NULL AND `DS_Processo` IS NOT NULL AND `NM_Processo` != "" AND `DS_Processo` != "" AND `NM_Processo` != "N/A" AND `DS_Processo` != "N/A"


### Executando a consulta SQL gerada

Agora saberemos se o objetvo foi alcançado. Nada de LLM, somente a boa e velha **QUERY**.

In [51]:
def executar_sql(consulta: str) -> dict:
    """Executa a consulta SQL e retorna os resultados."""   
    
    if consulta == "NOT_RELEVANT":
        return {"resultados": "NOT_RELEVANT"}

    try:
        resultados = db_manager.execute_query(consulta)
        return {"resultados": resultados}
    except Exception as e:
        return {"erro": str(e)}

In [52]:
resultados = executar_sql(consulta_sql_analisada['consulta_sql_analisada']) 
print(str(resultados))

{'resultados': [{'NM_Processo': 'Avaliação de Desempenho', 'DS_Processo': 'Coleta de feedbacks em andamento.'}]}


In [53]:
# Obtém os nomes das colunas
colunas = resultados["resultados"][0].keys()

# Imprime cabeçalho
print(" | ".join(colunas))
print("-" * (len(" | ".join(colunas)) + 5))

# Imprime os dados formatados
for linha in resultados["resultados"]:
    print(" | ".join(str(valor) for valor in linha.values()))

NM_Processo | DS_Processo
------------------------------
Avaliação de Desempenho | Coleta de feedbacks em andamento.


## Conclusão

Aparentemente os resultados foram bons, o processo criado abstrai bem a lógica de criação de um SQL, o LLM conseguiu realizar muito bem sua atividade, e, ao menos para mim, a resposta foi o esperado.

Claro que quanto mais os Agentes forem refinados, mais a resposta ficará melhor.

## Próximos Passos

Um ponto final só é o começo de uma nova frase, seguindo esse analogia, quero continuar evoluindo esse projeto. Algumas funcionalidades que irei implementar:

- Criar um Agente que cria uma resposta para o usuário.
- Criar um Grafo com lang-graf.
- Criar um Chat utilizando Chainlit.
- Quem sabe: Criar um Chatbot no Whatssapp com esse esquema.


## Minhas Limitações

Ainda tenho muitas duvidas quanto o processo de criar consultas a partir de texto natural, o TTQ(text to Query) ainda me parece um pouco complicado, mas isso vai melhorar ao longo do tempo.

- Como lidar com várias requisições vindas pelos Whatsapp
- Como manter o contexto em uma conversa, o usuário pode fazer uma pergunta e depois outra.

## Como Imagino Parte da Implementação

Já parti do principio que existe uma tabela com toda as informações necessárias. Por obvio ela deverá ser criada e as informações carregadas nela de alguma forma. Parte da lógica que usária é a seguinte:

- Criar a tabela que irá receber os processo.
- Cada processo criado dentro da ferramenta é responsável por registar seu progresso nessa tabela, pode ser com uma integração passagem de etapa.
  - Ao abrir o processo devesse registar o código dele nessa tabela
  - Ao passa cada atividade, ou ao menos as mais relevantes para consulta, buscar pela referencia do processo na tabela e atualizar as informações.
- Como algumas informações podem ser somente daquele processo, e não caber em um coluna genêrica, pode-se criar uma coluna que contenha um JSON com algumas informações especificas, essas informações podem servir de insumos para o Agente que irá criar a resposta.

Para lidar com várias requisições pode ser que de para usar uma fila, por exemplo Kafka.